In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import torch
from torch import nn

import warnings
warnings.filterwarnings('ignore')

In [2]:
def make_data(size: int, length: int):
    X = np.random.randint(0, 10, (size, length))
    y = np.zeros_like(X)
    for i in range(len(X)):
        y[i] = [X[i][0]] + [np.mod(x + X[i][0], 10) for x in X[i][1:]]
    return torch.tensor(X), torch.tensor(y)

make_data(size=2, length=3)

(tensor([[6, 2, 2],
         [9, 4, 1]]),
 tensor([[6, 8, 8],
         [9, 3, 0]]))

In [3]:
def train(model, X, y, epochs):
    for ep in range(epochs):
        model.train(True)
        start = time.time()
        train_loss = 0.0
        train_passed = 0

        queue = list(range(len(X)))
        np.random.shuffle(queue)
        for i in tqdm(queue):
            X_batch = X[i].to(device)
            Y_batch = y[i].to(device)

            optimizer.zero_grad()
            answers = model.forward(X_batch)
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    return model

In [4]:
def evaluate(model, X, y):
    model.train(False)
    for i in range(5):
        idx = np.random.choice(len(X))
        pred = model.forward(X[idx].to(device))
        print(f'Пример № {i+1}')
        print('Исходная последовательность:')
        print(y[idx])
        print('Предсказанная последовательность:')
        print(torch.argmax(pred, dim=1))
        print()

In [5]:
class Network(torch.nn.Module):
    def __init__(self, rnnClass):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(10, 32)
        self.rnn = rnnClass(32, 128)
        self.out = torch.nn.Linear(128, 10)

    def forward(self, seqs, state=None):
        x = self.embedding(seqs)
        x, s = self.rnn(x)
        return self.out(x)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
X, y = make_data(size=100000, length=50)

# RNN

In [8]:
model_rnn = Network(rnnClass=torch.nn.RNN)
model_rnn = model_rnn.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_rnn.parameters())

model_rnn = train(model_rnn, X, y, epochs=2)

100%|██████████| 100000/100000 [03:30<00:00, 475.10it/s]



Epoch 0. Time: 210.494, Train loss: 1.597


100%|██████████| 100000/100000 [03:27<00:00, 482.02it/s]


Epoch 1. Time: 207.479, Train loss: 0.275


In [9]:
evaluate(model_rnn, X, y)

Пример № 1
Исходная последовательность:
tensor([9, 2, 0, 0, 9, 3, 7, 4, 8, 6, 3, 8, 3, 7, 4, 5, 1, 5, 5, 5, 8, 6, 8, 1,
        5, 1, 9, 0, 0, 4, 6, 5, 8, 6, 6, 9, 0, 5, 0, 6, 7, 3, 5, 4, 8, 4, 4, 2,
        3, 1])
Предсказанная последовательность:
tensor([9, 2, 0, 0, 9, 3, 7, 4, 8, 6, 3, 8, 3, 7, 4, 5, 1, 5, 5, 5, 8, 6, 8, 1,
        5, 1, 9, 0, 0, 4, 6, 5, 8, 6, 6, 9, 0, 5, 0, 6, 7, 3, 5, 4, 8, 4, 4, 2,
        3, 1], device='cuda:0')

Пример № 2
Исходная последовательность:
tensor([6, 8, 9, 4, 5, 8, 6, 9, 3, 5, 3, 7, 8, 1, 1, 9, 3, 7, 6, 7, 5, 7, 2, 9,
        0, 0, 3, 4, 0, 2, 5, 1, 2, 6, 2, 3, 8, 9, 5, 4, 5, 3, 1, 6, 5, 6, 6, 6,
        2, 0])
Предсказанная последовательность:
tensor([6, 8, 9, 4, 5, 8, 6, 9, 3, 5, 3, 7, 8, 1, 1, 9, 3, 7, 6, 7, 5, 7, 2, 9,
        0, 0, 3, 4, 0, 2, 5, 1, 2, 6, 2, 3, 8, 9, 5, 4, 5, 3, 1, 6, 5, 6, 6, 6,
        2, 0], device='cuda:0')

Пример № 3
Исходная последовательность:
tensor([6, 2, 1, 0, 3, 5, 5, 9, 4, 6, 7, 2, 1, 3, 1, 3, 0, 0, 1, 3, 9, 7, 3,

# LSTM

In [10]:
model_lstm = Network(rnnClass=torch.nn.LSTM)
model_lstm = model_lstm.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_lstm.parameters())

model_lstm = train(model_lstm, X, y, epochs=2)

100%|██████████| 100000/100000 [03:45<00:00, 443.67it/s]



Epoch 0. Time: 225.403, Train loss: 0.112


100%|██████████| 100000/100000 [03:43<00:00, 447.12it/s]


Epoch 1. Time: 223.676, Train loss: 0.000


In [11]:
evaluate(model_lstm, X, y)

Пример № 1
Исходная последовательность:
tensor([6, 1, 8, 5, 4, 4, 5, 6, 2, 5, 4, 4, 1, 5, 4, 3, 3, 1, 7, 9, 1, 2, 4, 1,
        8, 4, 4, 6, 4, 8, 5, 1, 1, 1, 0, 4, 5, 6, 0, 4, 7, 3, 9, 7, 1, 0, 8, 6,
        7, 0])
Предсказанная последовательность:
tensor([6, 1, 8, 5, 4, 4, 5, 6, 2, 5, 4, 4, 1, 5, 4, 3, 3, 1, 7, 9, 1, 2, 4, 1,
        8, 4, 4, 6, 4, 8, 5, 1, 1, 1, 0, 4, 5, 6, 0, 4, 7, 3, 9, 7, 1, 0, 8, 6,
        7, 0], device='cuda:0')

Пример № 2
Исходная последовательность:
tensor([5, 8, 7, 4, 1, 8, 4, 2, 8, 7, 2, 9, 0, 3, 5, 7, 8, 3, 1, 6, 1, 1, 8, 8,
        4, 6, 5, 0, 4, 3, 3, 8, 0, 0, 8, 5, 0, 4, 8, 4, 3, 1, 0, 3, 9, 4, 3, 3,
        6, 7])
Предсказанная последовательность:
tensor([5, 8, 7, 4, 1, 8, 4, 2, 8, 7, 2, 9, 0, 3, 5, 7, 8, 3, 1, 6, 1, 1, 8, 8,
        4, 6, 5, 0, 4, 3, 3, 8, 0, 0, 8, 5, 0, 4, 8, 4, 3, 1, 0, 3, 9, 4, 3, 3,
        6, 7], device='cuda:0')

Пример № 3
Исходная последовательность:
tensor([7, 8, 3, 7, 5, 2, 1, 9, 7, 0, 3, 8, 7, 4, 4, 6, 9, 5, 7, 1, 0, 0, 5,

# GRU

In [12]:
model_gru = Network(rnnClass=torch.nn.GRU)
model_gru = model_gru.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_gru.parameters())

model_gru = train(model_gru, X, y, epochs=2)

100%|██████████| 100000/100000 [03:33<00:00, 467.36it/s]



Epoch 0. Time: 213.983, Train loss: 0.096


100%|██████████| 100000/100000 [03:34<00:00, 466.35it/s]


Epoch 1. Time: 214.448, Train loss: 0.001


In [13]:
evaluate(model_gru, X, y)

Пример № 1
Исходная последовательность:
tensor([6, 9, 6, 1, 8, 1, 0, 5, 4, 3, 7, 3, 4, 4, 1, 5, 4, 3, 5, 0, 0, 1, 4, 5,
        1, 6, 3, 8, 3, 4, 0, 1, 3, 6, 5, 1, 9, 0, 0, 5, 4, 6, 2, 5, 3, 9, 2, 0,
        8, 3])
Предсказанная последовательность:
tensor([6, 9, 6, 1, 8, 1, 0, 5, 4, 3, 7, 3, 4, 4, 1, 5, 4, 3, 5, 0, 0, 1, 4, 5,
        1, 6, 3, 8, 3, 4, 0, 1, 3, 6, 5, 1, 9, 0, 0, 5, 4, 6, 2, 5, 3, 9, 2, 0,
        8, 3], device='cuda:0')

Пример № 2
Исходная последовательность:
tensor([4, 4, 2, 5, 0, 3, 4, 7, 0, 7, 1, 9, 3, 3, 1, 1, 5, 5, 7, 7, 8, 8, 9, 2,
        1, 2, 5, 7, 4, 4, 3, 2, 3, 4, 2, 0, 3, 8, 8, 0, 1, 1, 9, 4, 2, 6, 1, 6,
        6, 3])
Предсказанная последовательность:
tensor([4, 4, 2, 5, 0, 3, 4, 7, 0, 7, 1, 9, 3, 3, 1, 1, 5, 5, 7, 7, 8, 8, 9, 2,
        1, 2, 5, 7, 4, 4, 3, 2, 3, 4, 2, 0, 3, 8, 8, 0, 1, 1, 9, 4, 2, 6, 1, 6,
        6, 3], device='cuda:0')

Пример № 3
Исходная последовательность:
tensor([7, 8, 3, 2, 0, 8, 7, 8, 8, 5, 6, 2, 8, 2, 0, 7, 5, 9, 5, 2, 9, 4, 3,